Testing shortcut keys

In [6]:
#this is code by default

This is markdown - type esc-m.

This code generates a simulated series of weather data from the Markov chain defined in Probabilistic Robotics, Thrun et al, Chapter 2, Problem 2.
The n'th column contains simulated weather n days following day 1 (on which the weather is assumed sunny).

In [11]:
#compare the results to the stationary distribution 
# calculated in closed form 
# (by calculating the 1-eigenvector of the transition matrix)
stationary_dist = (9.0/14.0,2.0/7.0, 1.0/14.0)
print(stationary_dist)

(0.6428571428571429, 0.2857142857142857, 0.07142857142857142)


In [7]:
from numpy.random import multinomial
import numpy as np
import collections

def run_sim(numdays):
    '''
    Generates a simulated series of weather data from the Markov chain defined in Probabilistic Robotics, Chapter 2, Problem 2.
    The n'th column contains simulated weather n days following day 1 (on which the weather is sunny).
    '''
    sunny = 1
    cloudy = 2
    rainy = 3
    def get_daytype(mvals):
        if mvals[0]:
            return sunny
        elif mvals[1]:
            return cloudy
        elif mvals[2]:
            return rainy
        else:
            raise Exception("mvals is all zeros")
    seq = np.zeros(numdays)
    tomorrow = 0
    today = sunny
    seq[0] = sunny
    for ii in range(1,numdays):
        if today is sunny:
            tomorrow = multinomial(1, [0.8,0.2,0.0])
        elif today is cloudy:
            tomorrow = multinomial(1, [.4,.4,.2])
        elif today is rainy:
            tomorrow = multinomial(1, [.2,.6,.2])
        else: 
            print("today is {}".format(today))
            raise Exception('hey')
        #print(get_daytype(tomorrow))
        seq[ii] = int(get_daytype(tomorrow))
        today = get_daytype(tomorrow)
    #print(seq)
    return seq


In [8]:
# Generates numsims sequences of weather for numdays following a sunny day
numsims = 2000
numdays = 4
results = np.zeros((numsims,numdays))
for ii in range(numsims): 
    results[ii,]=run_sim(numdays)

In [9]:
# Uses the previously simulated data to calculate 
# estimated probabilities of weather on each day for numdays following a sunny day
# You can see each day's probabilities starting to converge to the stationary distribution
for ii in range(1,numdays):
    nextday = results[:,ii]
    nextday_counts = collections.Counter(nextday)
    for key in nextday_counts: 
        nextday_counts[key] = nextday_counts[key]/float(numsims)
    print("Day {}:\n{}".format(ii,nextday_counts))

Day 1:
Counter({1.0: 0.7915, 2.0: 0.2085})
Day 2:
Counter({1.0: 0.6985, 2.0: 0.264, 3.0: 0.0375})
Day 3:
Counter({1.0: 0.657, 2.0: 0.28, 3.0: 0.063})


In [12]:
# run independent sets of sims numsets times to see if probabilities stabilize
numsets = 4
numdays=10 # number of days in series
numsims = 20000 # number of simulations to run

for ii in range(numsets):
    results = np.zeros((numsims,numdays))
    for ii in range(numsims): 
        results[ii,]= run_sim(numdays)
    
    # estimate the probabilities for the last day in the simulated series. 
    lastday = collections.Counter(results[:,numdays-1])
    for key in lastday:
        lastday[key]=lastday[key]/float(numsims)

    # estimate the probabilities for the next to last day in the simulated series.     
    next2lastday = collections.Counter(results[:,numdays-2])
    for key in next2lastday:
        next2lastday[key]=next2lastday[key]/float(numsims)
    print("last day count\n{}".format(lastday))
    print("next to last day count\n{}".format(next2lastday))
    print('---------------------')

last day count
Counter({1.0: 0.64675, 2.0: 0.28145, 3.0: 0.0718})
next to last day count
Counter({1.0: 0.64785, 2.0: 0.28285, 3.0: 0.0693})
---------------------
last day count
Counter({1.0: 0.6494, 2.0: 0.27855, 3.0: 0.07205})
next to last day count
Counter({1.0: 0.6507, 2.0: 0.2809, 3.0: 0.0684})
---------------------
last day count
Counter({1.0: 0.6421, 2.0: 0.28615, 3.0: 0.07175})
next to last day count
Counter({1.0: 0.64295, 2.0: 0.28495, 3.0: 0.0721})
---------------------
last day count
Counter({1.0: 0.64035, 2.0: 0.28805, 3.0: 0.0716})
next to last day count
Counter({1.0: 0.6373, 2.0: 0.29325, 3.0: 0.06945})
---------------------
